In [77]:
#import gensim.function

text=open('Стихи.txt','r').readlines()
text = [i for i in text if i != '' and 'Стихотворения' not in i]

In [78]:
text = [i.strip() for i in text]

In [79]:
text = [i for i in text if i != '' and 'Стихотворения' not in i]
text = text[:len(text)-243]
data=''.join(text)

In [80]:
len(data)

603368

In [81]:
data = data.replace('\xa0',' ')

for number in '123456789':
    data=data.replace(number,'0')
for english_letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    data=data.replace(english_letter, '')
for english_letter in 'abcdefghijklmnopqrstuvwzyz':
    data=data.replace(english_letter, '')    

In [82]:
b=set(data)
c=dict()
for symbol in b:
    c[symbol] = data.count(symbol)
import operator
c1 = sorted(c.items(), key=operator.itemgetter(1))

In [83]:
c1

[('„', 1),
 ('Й', 2),
 ('Щ', 7),
 ('_', 7),
 ('/', 11),
 ('Ю', 30),
 ('x', 37),
 ('Ц', 98),
 ('ъ', 101),
 ('Ш', 103),
 ("'", 128),
 ('(', 137),
 (')', 138),
 ('Э', 145),
 ('Ж', 160),
 ('«', 169),
 ('»', 169),
 ('#', 186),
 ('&', 186),
 ('Ф', 200),
 ('Х', 263),
 ('ё', 299),
 ('Е', 345),
 ('э', 380),
 ('-', 426),
 ('ф', 434),
 (']', 449),
 ('[', 450),
 ('Р', 474),
 ('Ч', 489),
 ('Л', 493),
 ('>', 504),
 ('<', 506),
 ('Я', 577),
 ('У', 584),
 ('"', 622),
 ('А', 624),
 ('Г', 680),
 ('З', 700),
 ('Б', 711),
 ('—', 715),
 ('*', 769),
 ('Д', 879),
 ('0', 896),
 ('–', 904),
 ('…', 920),
 ('М', 1052),
 ('?', 1139),
 (':', 1195),
 ('щ', 1214),
 ('О', 1353),
 ('Т', 1418),
 ('К', 1564),
 ('!', 1593),
 (';', 1750),
 ('С', 1976),
 ('П', 2220),
 ('ц', 2230),
 ('В', 2456),
 ('Н', 2494),
 ('И', 2956),
 ('ш', 4033),
 ('ю', 4386),
 ('х', 4900),
 ('.', 5011),
 ('ж', 5121),
 ('ч', 5844),
 ('з', 7547),
 ('г', 8269),
 ('б', 8544),
 ('я', 9137),
 ('ь', 10095),
 ('п', 10670),
 ('й', 10745),
 ('ы', 12039),
 ('к

In [84]:
Alphabet =''.join(c.keys())
Alphabet
#We don't delete any of the chars as now

'э!Р"ЙнМьН:ЗБъЖкКГФ.Щёгы,«уУюИХ<-Пд»похфЦОщxЛшЮ>йСДШ\'#–&в)аЕЧВ;рб0чТ„яз](цел_…?и сЭжЯ—[/мА*т'

In [125]:
return_first=True
def string_vectorizer(string, alphabet=Alphabet):
    vector = [[0 if char != letter else 1 for char in Alphabet] 
                  for letter in string]
    return vector
def vector_stringizer(vector_bag, alphabet = Alphabet):
    answ=''
    if return_first:
        for i in range(vector_bag.shape[0]):
            answ+=Alphabet[vector_bag[i,:].index(max(vector_bag))]
    else:
        for i in range(vector_bag.shape[0]):
            index = vector_bag[i:,].argsort()[-2:][0]
            answ+=Alphabet[vector_bag[i,:].index(max(vector_bag))]
    return answ
data_onehot = string_vectorizer(data)

KeyboardInterrupt: 

In [86]:
len(data_onehot)

595992

In [134]:
import keras
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from scipy import sparse

In [110]:
window_len = 5

In [112]:
def make_dataset(data_onehot):
    X_total,Y_total=[],[]
    for i in tqdm(range(window_len, len(data_onehot))):
        X_total.append(data_onehot[i-window_len:i])
        Y_total.append(data_onehot[i])
    return np.array(X_total), np.array(Y_total)
X_total, Y_total=make_dataset(data_onehot)

100%|██████████████████████████████| 595987/595987 [00:01<00:00, 304541.13it/s]


In [117]:
X_total.shape

(595987, 5, 91)

In [118]:
X_train,Y_train,X_test,Y_test = train_test_split(X_total, Y_total, test_size=0.15, random_state = 1)
del X_total
del  Y_total

In [141]:
X_train.shape

(506588, 5, 91)

In [139]:
model = Sequential()
model.add(LSTM(window_len+1, input_shape=(window_len, len(Alphabet)), return_sequences=True))
model.add(Flatten())
model.add(Dense(len(Alphabet)))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()
model.fit(X_train,Y_train, epochs=100,validation_data = (X_test, Y_test), batch_size=10, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 5, 6)              2352      
_________________________________________________________________
flatten_4 (Flatten)          (None, 30)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 91)                2821      
Total params: 5,173
Trainable params: 5,173
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking target: expected dense_4 to have 2 dimensions, but got array with shape (89399, 5, 91)